# UNET_ADD_SKIP
The following notebook implements the best unet architecture we were able to achieve. Images are split in tiles for training, each with size 512x512. This architecture uses two kind of connections between the five blocks on whic is built:
* __ADD_Connections__: allow to concatenate "vertically" consecutive blocks by adding the MaxPooling layer of the considered block with the one of the previous
* __SKIP_Connections__: allow to concatenate "in parallel" blocks as described in the paper describing the unet architecture

The model obtained is compiled with __Adam__ optimizer and two metrics: the __Accuracy__ and the __Intersection_Over_the_Union__. The loss used is a __weighted_categorical_cross_entropy__ which tries to give more importance to the classes that are less distinguishable in the images. To do so, respectively for __Background__, __Crop__ and __Weed__, we assigned the weights: 0.5, 1 and 1.5.

## Notebook Settings

This notebook was used to generate the best models trained on the following datasets:
* __Bipbip Haricot__
* __Pead Haricot__
* __Weedelec Haricot__

In order to select the correct dataset fill with a boolean variable the elements of the __bool_arr__ list in the next cell.

In [ ]:
import os

import tensorflow as tf
import numpy as np

SEED = 1234
img_h = 512
img_w = 512
bs = 8
lr = 1e-3
num_epochs = 100
patience = 15
num_classes = 3
debug_mode = False # true if you wanna use the reduced dataset, with only 3 imgs per folder

# kfold
val_split_perc = 0.2
k = 3
enable_kfold = False
if not enable_kfold:
  k = int(1 / val_split_perc)

# boolean flags
# choose here what to use for training
bool_arr = []
bool_arr.append([True, "Bipbip", "Haricot", ".jpg"])
bool_arr.append([False, "Bipbip", "Mais", ".jpg"])
bool_arr.append([False, "Pead", "Haricot", ".jpg"])
bool_arr.append([False, "Pead", "Mais", ".jpg"])
bool_arr.append([False, "Roseau", "Haricot", ".jpg"])
bool_arr.append([False, "Roseau", "Mais", ".jpg"])
bool_arr.append([False, "Weedelec", "Haricot", ".jpg"])
bool_arr.append([False, "Weedelec", "Mais", ".jpg"])

# choose here what to use for testing
bool_arr_test = []
bool_arr_test.append([True, "Bipbip", "Haricot", ".jpg"])
bool_arr_test.append([True, "Bipbip", "Mais", ".jpg"])
bool_arr_test.append([True, "Pead", "Haricot", ".jpg"])
bool_arr_test.append([True, "Pead", "Mais", ".jpg"])
bool_arr_test.append([True, "Roseau", "Haricot", ".png"])
bool_arr_test.append([True, "Roseau", "Mais", ".png"])
bool_arr_test.append([True, "Weedelec", "Haricot", ".jpg"])
bool_arr_test.append([True, "Weedelec", "Mais", ".jpg"])

model_name = 'final_unet_SKIP_ADD'

tf.random.set_seed(SEED)

### Mount Google Drive and Unzip the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

For debugging purposes we first used a reduced dataset and then moved to the one containing the tiles of all the images called __Development_Dataset_512__

In [ ]:
cwd = os.getcwd() # should be /content
dataset_version = 'Reduced_Development_Dataset' if debug_mode else 'Final_Dataset_512'

if debug_mode:
  if not os.path.exists(os.path.join(cwd, dataset_version)):
    !unzip '/content/drive/My Drive/challenge2/dataset/Reduced_Development_Dataset.zip'
else:
  if not os.path.exists(os.path.join(cwd, dataset_version)):
    !unzip '/content/drive/My Drive/challenge2/dataset/Final_Dataset_512.zip'

## Data Preparation
Once we specify in the first cell of the __Notebook_Settings__ paragraph the images we want to train our network with, we use the pandas dataframes for managing them.

In [ ]:
import pandas as pd

filenames_images = []
filenames_masks = []

base_folder = os.path.join(cwd, dataset_version, "Training")

for i in range(0,8):
  if bool_arr[i][0]:
    bf = []
    base_curr = os.path.join(base_folder, bool_arr[i][1], bool_arr[i][2])
    fn_images = [x for x in os.listdir(os.path.join(base_curr, "Images"))]
    fn_images.sort()
    fn_masks = [x for x in os.listdir(os.path.join(base_curr, "Masks"))]
    fn_masks.sort()

    for index, value in enumerate(fn_images):
      fn_images[index] = os.path.join(base_curr, "Images", value)

    for index, value in enumerate(fn_masks):
      fn_masks[index] = os.path.join(base_curr, "Masks", value)

    filenames_images += fn_images
    filenames_masks += fn_masks

base_folder = os.path.join(cwd, 'Final_Dataset_512', "Training")

for i in range(0,8):
  if bool_arr[i][0]:
    bf = []
    base_curr = os.path.join(base_folder, bool_arr[i][1], bool_arr[i][2])
    fn_images = [x for x in os.listdir(os.path.join(base_curr, "Images"))]
    fn_images.sort()
    fn_masks = [x for x in os.listdir(os.path.join(base_curr, "Masks"))]
    fn_masks.sort()

    for index, value in enumerate(fn_images):
      fn_images[index] = os.path.join(base_curr, "Images", value)

    for index, value in enumerate(fn_masks):
      fn_masks[index] = os.path.join(base_curr, "Masks", value)

    filenames_images += fn_images
    filenames_masks += fn_masks

data = pd.DataFrame(columns=["images", "masks"])
data["images"] = filenames_images
data["masks"] = filenames_masks


### Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=30,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='reflect',                                          
                                            rescale=1./255)
    
    train_mask_data_gen = ImageDataGenerator(rotation_range=30,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='reflect')
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255)
    train_mask_data_gen = ImageDataGenerator()

# Create validation and test ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator()

## Unet Model
The following cell defines the method which builds the Unet architecture with add and skip connections as defined at the beginning. By passing the number of classes on which the problem is defined we are able to parametrize the model also for more classes.

In [ ]:
from tensorflow.keras.layers import *

def upsampleLayer(in_layer, concat_layer, input_size):
  '''
  Upsampling (=Decoder) layer building block
  Parameters
  ----------
  in_layer: input layer
  concat_layer: layer with which to concatenate
  input_size: input size fot convolution
  '''
  upsample = Conv2DTranspose(input_size, (2, 2), strides=(2, 2), padding='same')(in_layer) 
  
  upsample = concatenate([concat_layer, upsample])
  
  conv = Conv2D(input_size, (3, 3), activation='relu', padding='same')(upsample)
  conv = BatchNormalization()(conv)
  conv = Dropout(0.2)(conv)

  return conv

def create_model(num_classes):

  inputs_1 = tf.keras.Input((img_h, img_w, 3))

  # encoder
  e1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs_1)
  e1b = MaxPool2D(pool_size=(2, 2))(e1)

  e3 = Conv2D(64, (3, 3), activation='relu', padding='same')(e1b)
  e3b = MaxPool2D(pool_size=(2, 2))(e3)
  r3 = Conv2D(64, 1, strides=2, padding="same")(e1b)
  e3b = add([e3b, r3])

  e4 = Conv2D(128, (3, 3), activation='relu', padding='same')(e3b)
  e4b = MaxPool2D(pool_size=(2, 2))(e4)
  r4 = Conv2D(128, 1, strides=2, padding="same")(e3b)
  e4b = add([e4b, r4])

  e5 = Conv2D(256, (3, 3), activation='relu', padding='same')(e4b)
  e5b = MaxPool2D(pool_size=(2, 2))(e5)
  r5 = Conv2D(256, 1, strides=2, padding="same")(e4b)
  e5b = add([e5b, r5])

  # bottleneck
  e6 = Conv2D(512, (3, 3), activation='relu', padding='same')(e5b)
  e6b = MaxPool2D(pool_size=(2, 2))(e6)
  r6 = Conv2D(512, 1, strides=2, padding="same")(e5b)
  e6b = add([e6b, r6])

  # decoder
  d1 = upsampleLayer(in_layer=e6b, concat_layer=e5b, input_size=256)
  rd1 = UpSampling2D(2)(e6b)
  rd1 = Conv2D(256, 1, padding="same")(rd1)
  d1 = add([d1, rd1])

  d2 = upsampleLayer(in_layer=d1, concat_layer=e4b, input_size=128)
  rd2 = UpSampling2D(2)(d1)
  rd2 = Conv2D(128, 1, padding="same")(rd2)
  d2 = add([d2, rd2])

  d3 = upsampleLayer(in_layer=d2, concat_layer=e3b, input_size=64)
  rd3 = UpSampling2D(2)(d2)
  rd3 = Conv2D(64, 1, padding="same")(rd3)
  d3 = add([d3, rd3])

  d4 = upsampleLayer(in_layer=d3, concat_layer=e1b, input_size=32)
  rd4 = UpSampling2D(2)(d3)
  rd4 = Conv2D(32, 1, padding="same")(rd4)
  d4 = add([d4, rd4])

  d5 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(d4)
  d5 = Conv2D(16, (3, 3), activation='relu', padding='same')(d5)
  d5 = BatchNormalization()(d5)
  d5 = Dropout(0.2)(d5)
  rd5 = UpSampling2D(2)(d4)
  rd5 = Conv2D(16, 1, padding="same")(rd5)
  d5 = add([d5, rd5])

  outputs = Conv2D(num_classes, (1, 1), activation='softmax')(d5)

  model = tf.keras.Model(inputs=inputs_1, outputs=outputs)
  
  return model
  

## Compiling Methods
For the multiclass segmentation problem we are tackling we need to specify the __IoU__ metric and a specific loss function in order to balance the importance of the different classes.

### Intersection Over Union

In [ ]:
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,num_classes): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

### Weighted Categorical Crossentropy Loss Function

In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)

    def loss(y_true, y_pred):
      #print("y_true shape={}\ny_pred shape={}\n{}".format(y_true.shape, y_pred.shape, y_true[:,:,:]))  
      # scale predictions so that the class probas of each sample sum to 1
      y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
      # clip to prevent NaN's and Inf's
      y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
      #trasformo da sparse representation a onehot representation per calcolare la loss
      #one_hot_encoding = np.zeros((y_true.shape[0], y_true.shape[1], y_true.shape[2], y_pred.shape[3]))
      #one_hot_encoding = K.zeros((y_true.shape[0], y_true.shape[1], y_true.shape[2], y_pred.shape[3]))
      one_hot_encoding = (tf.cast(tf.reduce_any(y_true == 0, axis=-1, keepdims=True), tf.float32)*tf.constant([1.,0.,0.]) +
                          tf.cast(tf.reduce_any(y_true == 1, axis=-1, keepdims=True), tf.float32)*tf.constant([0.,1.,0.])+
                          tf.cast(tf.reduce_any(y_true == 2, axis=-1, keepdims=True), tf.float32)*tf.constant([0.,0.,1.]))
                    

      # calc
      # se non fosse one hot avrei tipo [[0,1,0],[0,0,1]] * [[a,b,c], [d,e,f]] = [[a,b,c], [2d,2e,2f]], ma io voglio [[0,b,0], [0,0,f]]
      #1 2 
      loss = one_hot_encoding * K.log(y_pred) * weights
      loss = -K.sum(loss, -1)
      return loss
    
    return loss

## Prepare for Training

### Callbacks
The model is trained with 3 different callbacks that help both in storing the checkpoints containing the best weights found and in facing overfitting. The callbacks used are:
* __Checkpoint_Callback__
* __Learning_Rate_Scheduler_Callback__
* __Early_Stopping_Callback__

In [ ]:
def get_callbacks(exp_dir):
  callbacks = []

  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                    monitor='val_loss',
                                                    mode='min', 
                                                    save_weights_only=False,
                                                    save_best_only=True,
                                                    verbose=0)  
  callbacks.append(ckpt_callback)

  # learning rate scheduler callback
  def scheduler(epoch, lr):
    if epoch < 10:
      return lr
    else:
      return lr * tf.math.exp(-0.1)

  lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

  lr_scheduler = True
  if lr_scheduler:
    lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    callbacks.append(lr_callback)

  # Early Stopping
  # --------------
  early_stop = True
  if early_stop:
      es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                    patience=patience, 
                                                    restore_best_weights=True)
      callbacks.append(es_callback)

  return callbacks

### Save all the Hyperparameters
For hyperparameters tuning for each of the model tested we write a file containing all its hyperparameters.

In [ ]:
from datetime import datetime

# put here the path where you want to save the checkpoints of your model
exps_dir = '/content/drive/MyDrive/challenge2/2_phase/final_experiments'

if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

file1 = open(os.path.join(exp_dir, "parameters.txt"),"w")

file1.write('SEED = ' + str(SEED))
file1.write('\nimg_w = ' + str(img_w))
file1.write('\nimg_h = ' + str(img_h))
file1.write('\nbs = ' + str(bs))
file1.write('\nlr = ' + str(lr))
file1.write('\npatience = ' + str(patience))
file1.write('\nnum_epochs = ' + str(num_epochs))
file1.write('\nnum_classes = ' + str(num_classes))
file1.write('\ndebug_mode = ' + str(debug_mode))
file1.write('\nenable_kfold = ' + str(enable_kfold))
if enable_kfold:
  file1.write('\nk = ' + str(k))
else:
  file1.write('\nval_split_perc = ' + str(val_split_perc))

file1.write('\nBipbip Haricot = ' + str(bool_arr[0][0]))
file1.write('\nBipbip Mais = ' + str(bool_arr[1][0]))
file1.write('\nPead Haricot = ' + str(bool_arr[2][0]))
file1.write('\nPead Mais = ' + str(bool_arr[3][0]))
file1.write('\nRoseau Haricot = ' + str(bool_arr[4][0]))
file1.write('\nRoseau Mais = ' + str(bool_arr[5][0]))
file1.write('\nWeedelec Haricot = ' + str(bool_arr[6][0]))
file1.write('\nWeedelec Mais = ' + str(bool_arr[7][0]))

file1.close()

### Start Training
The prepare function is used for mapping the colors of the masks to the respective classes. Remember that we have:
* RGB: 0 0 0 - Target 0 (background)
* RGB: 254 124 18 - Target 0 (background)
* RGB: 255 255 255 - Target 1 (crop)
* RGB: 216 67 82 - Target 2 (weed)

In [ ]:
def prepare_target(x_, y_):
    return x_, (tf.cast(tf.reduce_any(y_ == 0, axis=-1, keepdims=True), tf.float32)*0 + 
                tf.cast(tf.reduce_any(y_ == 124, axis=-1, keepdims=True), tf.float32)*0 + 
                tf.cast(tf.reduce_any(y_ == 255, axis=-1, keepdims=True), tf.float32)*1 + 
                tf.cast(tf.reduce_any(y_ == 67, axis=-1, keepdims=True), tf.float32)*2) 

The training part allows for k-fold. In the case we are making cross validation we need first to specify the number of datasets k that we want to consider and then execute this cell. At the end we will have the best model among all the ones tested on the different datasets.

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=k, random_state=SEED, shuffle=True)

loop_iteration = 0

loss_arr = []
meanIoU_arr = []

for train_index, val_index in kfold.split(X=data["images"], y=data["masks"]):
  training_data = data.iloc[train_index]
  validation_data = data.iloc[val_index]

  #creation of the couple of generator for images and masks from training
  train_img_data_generator = train_img_data_gen.flow_from_dataframe(training_data,
                                                                    x_col = "images",
                                                                    shuffle = True,
                                                                    class_mode = None,
                                                                    target_size=(img_h, img_w),
                                                                    batch_size=bs,
                                                                    interpolation="nearest",
                                                                    seed=SEED)
  train_mask_data_generator = train_mask_data_gen.flow_from_dataframe(training_data,
                                                                      x_col = "masks",
                                                                      shuffle = True,
                                                                      class_mode = None,
                                                                      target_size=(img_h, img_w),
                                                                      batch_size=bs,
                                                                      interpolation="nearest",
                                                                      seed=SEED)
  train_gen = zip(train_img_data_generator, train_mask_data_generator)

  #creation of the couple of generators for images and masks for validation  
  valid_img_data_generator = valid_img_data_gen.flow_from_dataframe(validation_data,
                                                       x_col = "images",
                                                       shuffle = True,
                                                       class_mode = None,
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       interpolation="nearest",
                                                       seed = SEED)
  valid_mask_data_generator = valid_mask_data_gen.flow_from_dataframe(validation_data, 
                                                      x_col = "masks",
                                                      shuffle = True,
                                                      class_mode = None,
                                                      target_size=(img_h, img_w),
                                                      batch_size=bs,
                                                      interpolation="nearest",
                                                      seed = SEED)
  valid_gen = zip(valid_img_data_generator, valid_mask_data_generator)

  #######################

  train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
  train_dataset = train_dataset.map(prepare_target)
  train_dataset = train_dataset.repeat()

  valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
  valid_dataset = valid_dataset.map(prepare_target)
  valid_dataset = valid_dataset.repeat()

  #####################################################################################################################

  model = create_model(num_classes)
  model.summary()

  loss = weighted_categorical_crossentropy(np.asarray([0.5,1.,1.5], dtype=np.float32))
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
  metrics = ['accuracy', meanIoU]

  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

  callbacks = get_callbacks(exp_dir)

  history = model.fit(x=train_dataset,
              epochs=num_epochs,
              steps_per_epoch=len(train_img_data_generator),
              validation_data=valid_dataset,
              validation_steps=len(valid_img_data_generator), 
              callbacks=callbacks)

  minLoss = min(history.history['val_loss'])
  minLossIndex = history.history['val_loss'].index(minLoss)
  loss_arr.append(minLoss)
  meanIoU_arr.append(history.history['val_meanIoU'][minLossIndex])
  
  # print metrics to file
  with open(os.path.join(exp_dir, 'historySplit' + str(loop_iteration) + '.txt'), 'w') as f:
    for key in history.history.keys():
      print(str(key), file=f)
      print(history.history[key], file=f)
  
  if best_model_so_far == None:
    best_model_so_far = model

  if not enable_kfold:
    break
  
  loop_iteration += 1

with open(os.path.join(exp_dir, 'cv_results' + '.txt'), 'w') as f2:
  print("avg loss = {}".format(np.mean(loss_arr)), file=f2)
  print("avg meanIoU = {}".format(np.mean(meanIoU_arr)), file=f2)

## Prepare for Testing

### Find the Filenames
The following cell builds a list containing the filenames of all the images that we want to test on. In order to specify the team and plant we choose, use the __bool_arr_test__ list specified in the first cell of the notebook.

In [ ]:
test_images = []

base_folder = os.path.join(cwd, dataset_version, "Test_Final")

for i in range(0,8):
  if bool_arr_test[i][0]:
    team = []
    crop = []
    names = []
    base_curr = os.path.join(base_folder, bool_arr_test[i][1], bool_arr_test[i][2])
    fn_images = [x for x in os.listdir(os.path.join(base_curr, "Images"))]
    fn_images.sort()
    for entry in fn_images:
      names.append(entry[:-4])

    for j in range(0, len(fn_images)):
        team.append(bool_arr_test[i][1])
        crop.append(bool_arr_test[i][2])
    for index, value in enumerate(fn_images):
      fn_images[index] = os.path.join(base_curr, "Images", value)

    zipped_list = list(zip(fn_images, team, crop, names))

    test_images += zipped_list

### Testing Utils
The __rle_encode__ method is taken from the starting kit and is used to encode the prediction done on each image.

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

The following methods are used for tiling the test images once they have been loaded in memory. We need two functions to make the prediction in the correct way:
* __get_patches__: returns the tiles of an image in the specified size
* __reconstruct_from_patches__: builds back the image once we provide a stack of tiles

In [ ]:
# tiling utils
tile_size = 512

def get_patches(img_arr, size=256, stride=256):

    patches_list = []
    i_max = img_arr.shape[0] // stride
    j_max = img_arr.shape[1] // stride

    for i in range(i_max):
        for j in range(j_max):
            patches_list.append(
                img_arr[
                    i * stride : i * stride + size,
                    j * stride : j * stride + size
                ]
            )

    return np.stack(patches_list)

def reconstruct_from_patches(img_arr, org_img_size, stride, size):

    if img_arr.ndim == 3:
        img_arr = np.expand_dims(img_arr, axis=0)

    if size is None:
        size = img_arr.shape[1]

    if stride is None:
        stride = size

    nm_layers = img_arr.shape[3]

    i_max = (org_img_size[0] // stride) + 1 - (size // stride)
    j_max = (org_img_size[1] // stride) + 1 - (size // stride)

    total_nm_images = img_arr.shape[0] // (j_max * i_max)
    nm_images = img_arr.shape[0]

    averaging_value = size // stride
    images_list = []
    kk = 0
    for img_count in range(total_nm_images):
        img_bg = np.zeros(
            (org_img_size[0], org_img_size[1], nm_layers), dtype=img_arr[0].dtype
        )

        for i in range(i_max):
            for j in range(j_max):
                for layer in range(nm_layers):
                    img_bg[
                        i * stride : i * stride + size,
                        j * stride : j * stride + size,
                        layer,
                    ] = img_arr[kk, :, :, layer]

                kk += 1

        images_list.append(img_bg)
    return np.stack(images_list)

### Start Testing
Tiling is applied on test images at runtime. Each image is read with the respective filename, resized to the nearest multiple of the chosen tile size and cropped. Once we have the tiles we make the prediction on each of them, build a stack with the predictions and then reconstruct the full prediction with the same dimensions of the original image. At this point we can call the __rle_encode__ and insert in the submission dictionary the results for that image.

In [ ]:
import json
from PIL import Image

res_dir = '/content/drive/My Drive/challenge2/2_phase/Results'

# json generation
# ---------------
submission_dict = {}

for entry in test_images:
  
  image = Image.open(entry[0])
  width, height = image.size

  # resize image and create crops
  image = image.resize(((width // tile_size)*tile_size, (height // tile_size)*tile_size))
  img_arr = np.array(image)
  image_crops = get_patches(img_arr, size=tile_size, stride=tile_size)

  # prediction on each tile stacking each result
  tile_mask_list = []
  for i in range(len(image_crops)):
    tile_arr = image_crops[i]
    tile_arr = tile_arr * 1. / 255
    
    out_sigmoid = model.predict(x=tf.expand_dims(tile_arr, 0))
    
    predicted_class = tf.argmax(out_sigmoid, -1)
    predicted_class = predicted_class[0, ...]

    tile_mask_list.append(np.array(tf.expand_dims(predicted_class, axis=-1)))
  
  mask_crops = np.stack(tile_mask_list)

  # reconstruct and resize
  mask_reconstructed = reconstruct_from_patches(mask_crops, org_img_size=(image.height, image.width), stride=tile_size, size=tile_size)
  
  disegno = np.zeros((image.height, image.width , 3))
  disegno[np.where(mask_reconstructed[0,...,0] == 1)] = [255, 255, 255]
  disegno[np.where(mask_reconstructed[0,...,0] == 0)] = [0,0,0]
  disegno[np.where(mask_reconstructed[0,...,0] == 2)] = [216, 67, 82]

  imm = Image.fromarray(np.uint8(disegno)).resize((width, height))
  mask_arr = np.array(imm)

  new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

  new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
  new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2

  img_name = entry[3]

  submission_dict[img_name] = {}
  submission_dict[img_name]['shape'] = new_mask_arr.shape
  submission_dict[img_name]['team'] = entry[1]
  submission_dict[img_name]['crop'] = entry[2]
  submission_dict[img_name]['segmentation'] = {}

  # RLE encoding
  # crop
  rle_encoded_crop = rle_encode(new_mask_arr == 1)
  # weed
  rle_encoded_weed = rle_encode(new_mask_arr == 2)

  submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
  submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed


# Finally, save the results into the submission.json file
with open(os.path.join(res_dir, "submission.json"), 'w') as f:
  json.dump(submission_dict, f)